In [1]:
!pip install -qq adapters

In [ ]:
pip install accelerate -U

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/AdaptOrDie'

In [2]:
import json
import os
from IPython.display import Image

In [3]:
def load_json(path):
  with open(path) as f:
      data = [json.loads(line) for line in f]
  return data

def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# **RoBERTa Adapter-Base Model**

### **Loading model and adapter framework**

In [30]:
from transformers import RobertaTokenizer
from transformers import RobertaConfig
from adapters import AutoAdapterModel

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Pretrained RoBERTa model from Hugging Face
config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
# Add a new adapter to model
model.add_adapter("imbd_adapter", config="seq_bn")

# Add a matching classification head
model.add_classification_head(
    "imbd_adapter",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("imbd_adapter")

In [7]:
# # Add a new adapter to model
# model.add_adapter("helpfulness_adapter", config="seq_bn")

# # Add a matching classification head
# model.add_classification_head(
#     "helpfulness_adapter",
#     num_labels=2,
#     id2label={ 0: "👎", 1: "👍"}
#   )

# # Activate the adapter
# model.train_adapter("helpfulness_adapter")

# **Gurungaran DataSet**

## **Classification Review Datasets**

In [8]:
ds= {"imdb": {
        "data_dir": "https://s3-us-west-2.amazonaws.com/allennlp/dont_stop_pretraining/data/imdb/",
        "dataset_size": 20000
    },
    "amazon": {
        "data_dir": "https://s3-us-west-2.amazonaws.com/allennlp/dont_stop_pretraining/data/amazon/",
        "dataset_size": 115251
    }}

create_dir('datasets')
create_dir('datasets/classifier')
create_dir('datasets/classifier/imdb')
create_dir('datasets/classifier/amazon')

### **IMDB Dataset**

In [9]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/imdb/train.jsonl --output-dir 'datasets/classifier/imdb'
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/imdb/dev.jsonl --output-dir 'datasets/classifier/imdb'
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/imdb/test.jsonl --output-dir 'datasets/classifier/imdb'

100 25.6M  100 25.6M    0     0  20.8M      0  0:00:01  0:00:01 --:--:-- 20.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6651k  100 6651k    0     0  8678k      0 --:--:-- --:--:-- --:--:-- 8671k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31.4M  100 31.4M    0     0  19.6M      0  0:00:01  0:00:01 --:--:-- 19.6M


In [10]:
train_json = load_json("datasets/classifier/imdb/train.jsonl")
dev_json = load_json("datasets/classifier/imdb/dev.jsonl")
test_json = load_json("datasets/classifier/imdb/test.jsonl")
print("IMDB ds stat")
print("Train ds: {0}".format(len(train_json)))
print("Dev ds: {0}".format(len(dev_json)))
print("Test ds: {0}".format(len(test_json)))

IMDB ds stat
Train ds: 20000
Dev ds: 5000
Test ds: 25000


### **Helfulness Dataset**

In [11]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/train.jsonl --output-dir 'datasets/classifier/amazon'
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/dev.jsonl --output-dir 'datasets/classifier/amazon'
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/amazon/test.jsonl --output-dir 'datasets/classifier/amazon'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  100M  100  100M    0     0  21.9M      0  0:00:04  0:00:04 --:--:-- 21.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4523k  100 4523k    0     0  6043k      0 --:--:-- --:--:-- --:--:-- 6048k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.7M  100 21.7M    0     0  22.9M      0 --:--:-- --:--:-- --:--:-- 22.9M


In [12]:
train_json_h = load_json("datasets/classifier/amazon/train.jsonl")
dev_json_h = load_json("datasets/classifier/amazon/dev.jsonl")
test_json_h = load_json("datasets/classifier/amazon/test.jsonl")
print("Helfulness ds stat")
print("Train ds: {0}".format(len(train_json_h)))
print("Dev ds: {0}".format(len(dev_json_h)))
print("Test ds: {0}".format(len(test_json_h)))

Helfulness ds stat
Train ds: 115251
Dev ds: 5000
Test ds: 25000


### **Combining Datasets**

In [13]:
# Combining training datasets
json_files = ["datasets/classifier/imdb/train.jsonl", "datasets/classifier/imdb/dev.jsonl", "datasets/classifier/imdb/test.jsonl", "datasets/classifier/amazon/train.jsonl", "datasets/classifier/amazon/dev.jsonl", "datasets/classifier/amazon/test.jsonl"]
reviews_datasets = []

# Load each JSON file into a Python object.
for json_file in json_files:
  a = load_json(json_file)
  reviews_datasets.extend(a)

# Dump all the Python objects into a single JSON file.
with open("combined.json", "w") as f:
    json.dump(reviews_datasets, f, indent=4)

# **Training**

Continued pretraining of RoBERTa on two
categories of unlabeled data: (i) large corpora of
domain-specific text, and (ii) available unlabeled data associated with a given task


In [ ]:
!pip install -U datasets

In [14]:
# Encode dataset - need to tokenize json data
# Encoded dataset for transformer model - user Roberta Tokenizer
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

In [15]:
from datasets import Dataset
review_data = Dataset.from_list(reviews_datasets)

ArrowInvalid: Could not convert 'helpful' with type str: tried to convert to int64

In [16]:
# imdb
imdb_data = [train_json, dev_json, test_json]

imdb_combined = [x for xs in imdb_data for x in xs]

In [21]:
imdb_data = Dataset.from_list(imdb_combined)

In [23]:
imdb_data_train = Dataset.from_list(train_json)
# Encode the input data
imdb_data_train = imdb_data_train.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset_train = imdb_data_train.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
dataset_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [24]:
imdb_data_validate = Dataset.from_list(dev_json)
# Encode the input data
imdb_data_validate = imdb_data_validate.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset_val = imdb_data_validate.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
dataset_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [18]:
# Encode the input data
imdb_data = imdb_data.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = imdb_data.rename_column(original_column_name="label", new_column_name="labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

## **Domain-Adaptive**

In [19]:
# CLEAN MODEL
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Pretrained RoBERTa model from Hugging Face
config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
clean_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model has added adapters

### **Reviews Dataset**

In [28]:
# Train model
import numpy as np
from transformers import TrainingArguments, EvalPrediction, Trainer
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = Trainer(
    model=clean_model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_accuracy,
)

In [29]:
# TRAINING
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 600.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 115.06 MiB is free. Process 102016 has 14.63 GiB memory in use. Of the allocated memory 14.03 GiB is allocated by PyTorch, and 494.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Evaluate on validation split
trainer.evaluate()

## **Task-Adaptive**

### **Reviews Dataset**

In [ ]:
# Train model

# **Evaluation**

# **Adapter Training**

In [32]:
# Use Adapter Training Class
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_accuracy,
)

KeyError: "Column train not in the dataset. Current columns in the dataset: ['id', 'text', 'labels', 'input_ids', 'attention_mask']"

In [ ]:
# TRAINING
trainer.train()

In [ ]:
# Save trained adapter
model.save_adapter("./imdb_adapter", "imdb_adapter")

!ls -lh imdb_adapter

# **Adapter Evaluation**